import plotly.graph_objects as go

# Convert summary_df values to string (if not already) for table
table_values = []
for col in summary_df.columns:
    table_values.append(summary_df[col].astype(str).tolist())

# Define colors for main and compared country
colors = []
for col in summary_df.columns:
    if col == USER_COUNTRY:
        colors.append([USER_COLOR]*len(summary_df))
    elif col == HIGHLIGHT_COUNTRY:
        colors.append([HIGHLIGHT_COLOR]*len(summary_df))
    else:
        colors.append(["white"]*len(summary_df))

fig_table = go.Figure(data=[go.Table(
    header=dict(
        values=list(summary_df.columns),
        fill_color='lightgrey',
        align='center',
        font=dict(size=14, color='black')
    ),
    cells=dict(
        values=table_values,
        fill_color=colors,
        align='center',
        font=dict(color='black', size=12)
    )
)])

fig_table.update_layout(title="Country Debt Comparison Table")
fig_table.show()
